In [11]:
using DrWatson
@quickactivate "CVDAnthracyclines"
using JSON3
using HTTP
using OMOPCDMCohortCreator
using CSV
using DataFrames

import DBInterface:
    connect,
    execute
import LibPQ:
    Connection

conn = connect(
    Connection, 
    "host=localhost port=5432 dbname=synthea user=thecedarprince password=JJJOjjjo1*"
)    
GenerateDatabaseDetails(:postgresql, "omop")
GenerateTables(conn)


┌ Info: Global database dialect set to: postgresql
└ @ OMOPCDMCohortCreator /home/jakubmitura/.julia/packages/OMOPCDMCohortCreator/0Dq7P/src/generators.jl:150
┌ Info: Global schema set to: omop
└ @ OMOPCDMCohortCreator /home/jakubmitura/.julia/packages/OMOPCDMCohortCreator/0Dq7P/src/generators.jl:151
┌ Info: source_to_concept_map table generated internally
└ @ OMOPCDMCohortCreator /home/jakubmitura/.julia/packages/OMOPCDMCohortCreator/0Dq7P/src/generators.jl:173
┌ Info: payer_plan_period table generated internally
└ @ OMOPCDMCohortCreator /home/jakubmitura/.julia/packages/OMOPCDMCohortCreator/0Dq7P/src/generators.jl:173
┌ Info: vocabulary table generated internally
└ @ OMOPCDMCohortCreator /home/jakubmitura/.julia/packages/OMOPCDMCohortCreator/0Dq7P/src/generators.jl:173
┌ Info: note_nlp table generated internally
└ @ OMOPCDMCohortCreator /home/jakubmitura/.julia/packages/OMOPCDMCohortCreator/0Dq7P/src/generators.jl:173
┌ Info: procedure_occurrence table generated internally
└ @ OMOPCD

PostgreSQL connection (CONNECTION_OK) with parameters:
  user = thecedarprince
  password = ********************
  channel_binding = prefer
  dbname = synthea
  host = localhost
  port = 5432
  client_encoding = UTF8
  options = -c DateStyle=ISO,YMD -c IntervalStyle=iso_8601 -c TimeZone=UTC
  application_name = LibPQ.jl
  sslmode = prefer
  sslcompression = 0
  sslsni = 1
  ssl_min_protocol_version = TLSv1.2
  gssencmode = prefer
  krbsrvname = postgres
  target_session_attrs = any

In [57]:
using Pkg
# Pkg.add("FunSQL")

### Explore miocardial infarction codes
After manually got through all codes about miocardial infarction those significant have 1 in column "was"
those that are marked as subsequent has also 1 in column "subsequent"


In [12]:
miocard_path="/home/jakubmitura/projects/CVDAnthracyclines/data/ATLAS_miocardial_infarction.csv"
miocard_df = CSV.File(miocard_path) |> DataFrame
df=miocard_df[!,["Id","Name","was","Vocabulary"]]
df = df[.!ismissing.(df.was), :]
df = df[.!ismissing.(df.Id), :]
df_was = df[df.was .== 1, :]
# unique(df_was[!,"was"])
df_was[!,"Id"]

707-element Vector{Union{Missing, Int64}}:
 40398840
  1413806
 37163023
 37163022
 45766075
  3227072
  3173997
  4178129
  4267568
  3414739
        ⋮
 40394118
  3058027
 40394119
  3528915
 45443368
 40398889
 44795635
  3080303
  3057773

In [13]:
all_miocardial_infarction_ids=ConditionFilterPersonIDs(df_was[!,"Id"], conn)
miocardial_infarction_dates=ConditionFilterPersonIDs([1034820,40761337], conn)

# diseased_patients = Dict()
# for (idx, disease) in enumerate(df_was[!,"Id"])
#     ids = ConditionFilterPersonIDs(
#         [
#             d[:CONCEPT_ID] 
#             for d in disease_descendants[idx]
#             ], conn).person_id
#     push!(
#         diseased_patients, 
#         disease.name => 
#             (ids = ids, count = length(ids))
#         )
# end
length(all_miocardial_infarction_ids.person_id), length(miocardial_infarction_dates.person_id)

(2250, 0)

In [56]:
sql = From(procedure_occurrence) |>
Group() |>
Select(:first_year => Agg.min(Get.observation_period_end_date),
    :last_year => Agg.max(Get.observation_period_end_date)) |>
q -> render(q, dialect=OMOPCDMCohortCreator.dialect)

return String(sql)


UndefVarError: UndefVarError: `From` not defined

In [55]:
stmt = DBInterface.prepare(conn, """SELECT * FROM  omop.procedure_occurrence  LIMIT 1""") # prepare a sql statement against the connection; returns a statement object
results = DBInterface.execute(stmt) # execute a prepared statement; returns an iterator of rows (property-accessible & indexable)
result.column_names

2-element Vector{String}:
 "no_nulls"
 "yes_nulls"

In [50]:
stmt = DBInterface.prepare(conn, """SELECT * FROM  omop.procedure_occurrence  LIMIT 1""") # prepare a sql statement against the connection; returns a statement object
results = DBInterface.execute(stmt) # execute a prepared statement; returns an iterator of rows (property-accessible & indexable)
result.column_names
# rowid = DBInterface.lastrowid(results) # get the last row id of an INSERT statement, as supported by the database

# # example of using a query resultset
for row in results
    @show propertynames(row) # see possible column names of row results
    # row.col1 # access the value of a column named `col1`
    # print(row)
    # row[1] # access the first column in the row results
end

# OMOPCDMCohortCreator.render(results, dialect=OMOPCDMCohortCreator.dialect)

In [52]:
result.column_names


2-element Vector{String}:
 "no_nulls"
 "yes_nulls"

In [49]:
using LibPQ
table_data = LibPQ.column_names(result)
table_data

2-element Vector{String}:
 "no_nulls"
 "yes_nulls"

In [21]:
using DBInterface
sql =
DBInterface.From(condition_occurrence) |>
Where(Fun.in(df_was[!,"Id"])) |>
Select(df_was[!,"Id"]) |>
q -> render(q, dialect=OMOPCDMCohortCreator.dialect)

String(sql)

UndefVarError: UndefVarError: `From` not defined

In [5]:
drugs = [
    (name = "Daunorubicin", id = 1311799),
    (name = "Doxorubicin", id = 1338512),
    (name = "Epirubicin", id = 1344354),
    (name = "Idarubicin", id = 19078097),
    (name = "Mitoxantrone", id = 1309188),
    (name = "Valrubicin", id = 19012543),
    (name = "Ibrutinib", id = 44507848),
    (name = "Acalabrutinib", id = 792764),
    (name = "Zanubrutinib", id = 37497691),
]

9-element Vector{@NamedTuple{name::String, id::Int64}}:
 (name = "Daunorubicin", id = 1311799)
 (name = "Doxorubicin", id = 1338512)
 (name = "Epirubicin", id = 1344354)
 (name = "Idarubicin", id = 19078097)
 (name = "Mitoxantrone", id = 1309188)
 (name = "Valrubicin", id = 19012543)
 (name = "Ibrutinib", id = 44507848)
 (name = "Acalabrutinib", id = 792764)
 (name = "Zanubrutinib", id = 37497691)

Here are the diseases we are exploring:

In [21]:
diseases = [
    (name = "Myocardial Infarction", id = 4329847),
    (name = "Heart Failure", id = 316139),
]

2-element Vector{@NamedTuple{name::String, id::Int64}}:
 (name = "Myocardial Infarction", id = 4329847)
 (name = "Heart Failure", id = 316139)

For both these drugs and diseases, the IDs come from [ATLAS](https://atlas-demo.ohdsi.org/#/search), a tool to develop computable phenotype definitions (i.e. translating a disease phenotype definition to the same thing but in code).
How these IDs were found for a drug is as follows (using Doxorubicin as an example):

1. Search within ATLAS for Doxorubicin.

2. Click any Doxorubicin-related drug name.

3. Look through the related concepts or hierarchy (in the parent section) for the ingredient version of the drug.

4. Copy that ID for the ingredient version of Doxorubicin.

That is the ID we used here as it is the base for all other Doxorubicin medications.

Next, we can do something similar for Myocardial Infarction:

1. Search within ATLAS for Myocardial Infarction.

2. Click any Myocardial Infarction-related name.

3. Look through the hierarchy of this disease. 
The condition version we are looking for is that parent concepts are too broad for our condition and the current condition we select should have children that encompass all diseases we want (in this case, this is [recorded condition](https://atlas-demo.ohdsi.org/#/concept/4329847) is the perfect level of granularity).

4. Copy that ID for the ingredient version of Myocardial Infarction.

That is the ID we used here as it is the base for all other Myocardial Infarction-related disease conditions.

Next, we'll discuss how to create a query that includes the derivatives of the general concepts for the drugs and diseases we found.

### Generating Ad Hoc Definitions

Here, we programmatically query the public ATLAS instance that is running to get the children IDs of the general concepts (referred to as descendants in ATLAS).
In this case, we are, in a very ad hoc way, creating an implicit computable phenotype definition where we are looking for all drugs and diseases that are based off our general concepts.
This is great for rapid prototyping, but a phenotype definition must be scrutinized ruthlessly when moving towards a final version for patient analysis (see: [Zelko, Jacob S., et al. "Developing a Robust Computable Phenotype Definition Workflow to Describe Health and Disease in Observational Health Research." arXiv preprint arXiv:2304.06504 (2023).](https://arxiv.org/abs/2304.06504) which discussed more about this process).

This is how we do that for drugs:

In [7]:
drug_descendants = []
for drug in drugs
    path = "https://atlas-demo.ohdsi.org/WebAPI/vocabulary/ATLASPROD/concept/$(drug.id)/descendants/"
    descendants = 
        HTTP.get(path) |> 
        x -> String(x.body) |> 
        JSON3.read
    push!(drug_descendants, descendants)
end
drug_descendants


9-element Vector{Any}:
 JSON3.Object[{
                 "CONCEPT_ID": 41047666,
               "CONCEPT_NAME": "Daunorubicin 20 MG [Daunoblastin]",
           "STANDARD_CONCEPT": "S",
   "STANDARD_CONCEPT_CAPTION": "Standard",
             "INVALID_REASON": "V",
     "INVALID_REASON_CAPTION": "Valid",
               "CONCEPT_CODE": "OMOP2245628",
                  "DOMAIN_ID": "Drug",
              "VOCABULARY_ID": "RxNorm Extension",
           "CONCEPT_CLASS_ID": "Branded Drug Comp",
           "VALID_START_DATE": 1503532800000,
             "VALID_END_DATE": 4102358400000,
              "RELATIONSHIPS": [
                                 {
                                        "RELATIONSHIP_NAME": "Descendant",
                                    "RELATIONSHIP_DISTANCE": 1
                                 }
                               ],
       "RELATIONSHIP_CAPTION": "Descendant"
}, {
                 "CONCEPT_ID": 36893752,
               "CONCEPT_NAME": "Daunorubicin 2.14 MG

This is how we do that for diseases:

In [22]:
disease_descendants = []
for disease in diseases
    path = "https://atlas-demo.ohdsi.org/WebAPI/vocabulary/ATLASPROD/concept/$(disease.id)/descendants/"
    descendants = 
        HTTP.get(path) |> 
        x -> String(x.body) |> 
        JSON3.read
    push!(disease_descendants, descendants)
end
disease_descendants

2-element Vector{Any}:
 JSON3.Object[{
                 "CONCEPT_ID": 604425,
               "CONCEPT_NAME": "Acute myocardial infarction due to occlusion of circumflex branch of left coronary artery",
           "STANDARD_CONCEPT": "S",
   "STANDARD_CONCEPT_CAPTION": "Standard",
             "INVALID_REASON": "V",
     "INVALID_REASON_CAPTION": "Valid",
               "CONCEPT_CODE": "896689003",
                  "DOMAIN_ID": "Condition",
              "VOCABULARY_ID": "SNOMED",
           "CONCEPT_CLASS_ID": "Disorder",
           "VALID_START_DATE": 1612051200000,
             "VALID_END_DATE": 4102358400000,
              "RELATIONSHIPS": [
                                 {
                                        "RELATIONSHIP_NAME": "Descendant",
                                    "RELATIONSHIP_DISTANCE": 3
                                 }
                               ],
       "RELATIONSHIP_CAPTION": "Descendant"
}, {
                 "CONCEPT_ID": 45766075,
              

### Analyzing the OMOPCDM Database

Find users of the drugs we are interested in:

In [9]:
drug_users = Dict()
for (idx, drug) in enumerate(drugs)
    ids = OMOPCDMCohortCreator.DrugExposureFilterPersonIDs(
        [
            drug[:CONCEPT_ID] 
            for drug in drug_descendants[idx]
        ], conn).person_id
    push!(
        drug_users, 
        drug.name => 
            (ids = ids, count = length(ids))
    )
end

Find patients diagnosed with the conditions we are interested in:

In [10]:
diseased_patients = Dict()
for (idx, disease) in enumerate(diseases)
    ids = ConditionFilterPersonIDs(
        [
            d[:CONCEPT_ID] 
            for d in disease_descendants[idx]
            ], conn).person_id
    push!(
        diseased_patients, 
        disease.name => 
            (ids = ids, count = length(ids))
        )
end



Find the combination of patients diagnosed with the conditions we are interested in and taking the drugs we want to investigate:

In [11]:
condition_mix = Dict()
for pg in keys(diseased_patients)
    push!(condition_mix, pg => Dict())
    for du in keys(drug_users)
        mix = intersect(
            diseased_patients[pg].ids, 
            drug_users[du].ids
        )
        push!(
            condition_mix[pg], 
            du => (ids = mix, count = length(mix))
        )
    end
end

### Saving Results

In [12]:
open(datadir("exp_raw", "drug_user_counts.csv"), "w") do io
    write(io, "DRUG,COUNT\n")
    for du in keys(drug_users)
        write(io, "$du,$(drug_users[du].count)\n")
    end
end

In [13]:
open(datadir("exp_raw", "diseased_patient_counts.csv"), "w") do io
    write(io, "DISEASE,COUNT\n")
    for pg in keys(diseased_patients)
        write(io, "$pg,$(diseased_patients[pg].count)\n")
    end
end

In [14]:
open(datadir("exp_raw", "mix_counts.csv"), "w") do io
    write(io, "DISEASE,DRUG,COUNT\n")
    for disease in keys(condition_mix)
        for drug in keys(condition_mix[disease])
            write(
                io, 
                """
                $disease,
                $drug,
                $(condition_mix[disease][drug].count)\n
                """
            )
        end
    end
end

In [ ]:
You are sql expert; design query ; remember that all of the table are in "omop" database get information from 
table concept
we look for row with value of a query in "concept_id" column
    then print result from column_names
    ["concept_name","concept_code","concept_class_id"]

## References: